In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

import numpy as np
from collections import Counter
import os
from argparse import Namespace

In [3]:
train_file='metal_feed_to.txt'
seq_size=32
batch_size=64
embedding_size=64
lstm_size=64
gradients_norm=5
initial_words=['i', 'am']
predict_top_k=5
checkpoint_path='checkpoint'

# Cleaning data

In [4]:
def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r') as f:
        text = f.read()
    text = (text.strip().lower()
        .replace('\n', ' \n ')
        .replace('!', ' ! ')
        .replace('?', ' ? ')
        .replace(',', ' , ')
       .replace('  ', ' '))
    text = text.split(' ')

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text

# Batches

In [5]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

# LSTM module

In [6]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state
    
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

In [7]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer

In [8]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
    train_file, batch_size, seq_size)

net = RNNModule(n_vocab, seq_size,
                embedding_size, lstm_size)
net = net.to(device)

criterion, optimizer = get_loss_and_train_op(net, 0.01)

iteration = 0
for e in range(50):
        batches = get_batches(in_text, out_text, batch_size, seq_size)
        state_h, state_c = net.zero_state(batch_size)
        
        # Transfer data to GPU
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            
            # Tell it we are in training mode
            net.train()

            # Reset all gradients
            optimizer.zero_grad()

            # Transfer data to GPU
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss_value = loss.item()

            # Perform back-propagation
            #loss.backward()

            # Update the network's parameters
            optimizer.zero_grad()
            loss.backward()

            _ = torch.nn.utils.clip_grad_norm_(
                net.parameters(), gradients_norm)

            optimizer.step()
            
            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

            if iteration % 200 == 0:
                
                predict(device, net, initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)
                
                torch.save(net.state_dict(),
                           'LSTM_gener')
                
                words_dics = [seq_size, n_vocab, vocab_to_int, embedding_size, lstm_size, int_to_vocab]
                
                with open('dictionaries' + '.pkl', 'wb') as f:
                    pickle.dump(words_dics, f, pickle.HIGHEST_PROTOCOL)

Vocabulary size 90249
Epoch: 0/200 Iteration: 100 Loss: 6.442417621612549
Epoch: 0/200 Iteration: 200 Loss: 6.194726467132568
i am your will can be your soul 
 the can we will i will be your can 
 a soul is your am i can be the soul in me in your soul 
 we 
 i am me 
 you will the will can i 
 you know your the will the eyes of me in me 
 we can can you will you will i 
 the soul of the eyes ! 
 i 
 a will you know 
 we have you will be the time of the 
 i will 
 the am your can be the 

Epoch: 0/200 Iteration: 300 Loss: 5.748708724975586
Epoch: 0/200 Iteration: 400 Loss: 5.641971111297607
i am your will can be your soul 
 the can we will i will be your can 
 a soul is your am i can be the soul in me in your soul 
 we 
 i am me 
 you will the will can i 
 you know your the will the eyes of me in me 
 we can can you will you will i 
 the soul of the eyes ! 
 i 
 a will you know 
 we have you will be the time of the 
 i will 
 the am your can be the 
 you have you can have been in a new 

KeyboardInterrupt: 

# Continue training if the model is saved

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
    train_file, batch_size, seq_size)

net = RNNModule(n_vocab, seq_size,
                embedding_size, lstm_size)

net.load_state_dict(torch.load('LSTM_gener'))

net = net.to(device)

criterion, optimizer = get_loss_and_train_op(net, 0.01)

iteration = 0
for e in range(50):
        batches = get_batches(in_text, out_text, batch_size, seq_size)
        state_h, state_c = net.zero_state(batch_size)
        
        # Transfer data to GPU
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            
            # Tell it we are in training mode
            net.train()

            # Reset all gradients
            optimizer.zero_grad()

            # Transfer data to GPU
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss_value = loss.item()

            # Perform back-propagation
            #loss.backward()

            # Update the network's parameters
            optimizer.zero_grad()
            loss.backward()

            _ = torch.nn.utils.clip_grad_norm_(
                net.parameters(), gradients_norm)

            optimizer.step()
            
            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))